In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
jobs_df = pd.read_csv('data.csv')

In [ ]:
jobs_df.head(5)

In [ ]:
len(jobs_df)

In [ ]:
jobs_df.info()

In [ ]:
jobs_df.describe().astype(int)

In [ ]:
jobs_df.isna().sum()

In [ ]:
jobs_df.corr(numeric_only=True)

In [ ]:
jobs_df.hist(bins=100, figsize=(16, 8))
plt.show()

In [ ]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center')

In [ ]:
jobs_df['company size'].value_counts().sort_index().values

In [ ]:
jobs_df['company size'].value_counts().sort_index().plot.bar(title='Liczba ofert w zależności od rozmiaru firmy')
plt.xlabel("Rozmiar firmy")
plt.ylabel("Liczba ofert")
ax = plt.gca()

# Add the values on top of the bars
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x()+0.05, p.get_height()))

plt.show()
plt.show()

In [ ]:
jobs_df['company'].value_counts()

In [ ]:
jobs_df['location'].value_counts().plot.bar()

In [ ]:
jobs_df['technology'].value_counts().plot.bar()

In [ ]:
jobs_df['seniority'].value_counts().plot.bar()

In [ ]:
jobs_df.corr()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error

In [ ]:
jobs_df.dropna(subset=['location', 'technology', 'seniority']).isna().sum()

In [ ]:
jobs_df.info()

In [ ]:
target = 'salary employment min'

In [ ]:
jobs_filtered_df = jobs_df.dropna(subset=['location', 'technology', 'seniority', target])

In [ ]:
X = jobs_filtered_df.filter(regex='^(?!.*salary).*').filter(regex='^(?!.*company).*')
y = jobs_filtered_df[target]

In [ ]:
OrdinalEncoder().fit_transform(X['seniority'].to_numpy().reshape(-1, 1))

In [ ]:
OrdinalEncoder().fit_transform(X['technology'].to_numpy().reshape(-1, 1))

In [ ]:
OrdinalEncoder().fit_transform(X['location'].to_numpy().reshape(-1, 1))

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
pipeline = make_pipeline(OrdinalEncoder(), SimpleImputer(), StandardScaler(), LinearRegression())
pipeline

In [ ]:
cross_val_score(pipeline, X_train, y_train, cv=10, scoring=make_scorer(mean_absolute_error)).mean()

In [ ]:
np.sqrt(cross_val_score(pipeline, X_train, y_train, cv=10, scoring=make_scorer(mean_squared_error)).mean())